In [1]:
#import sklearn library
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#dictionary with usernames their watched web series and ratings out of 5
dataset={
        'Rahul': {'GitHub Documentation by FreeCodecamp': 5,
                  'ReactJS by lamadev': 3,
                  'NodeJs by Code with Harry': 2,
                  'Basic HTML CSS by code with harry': 3},
    
        'Rishabh': {'GitHub Documentation by FreeCodecamp': 5,
                    'Basic HTML CSS by code with harry': 3,
                    'NodeJs by Code with Harry': 3},
    
        'Sonali': {'GitHub Documentation by FreeCodecamp': 2,
                   'JavaScript by freecodecamp': 5,
                   'Web3 by edroh': 3,
                   'Basic HTML CSS by code with harry': 4},
    
        'Ritvik': {'JavaScript by freecodecamp': 5,
                   'Basic HTML CSS by code with harry': 4,
                   'Web3 by edroh': 4,},
    
       'Harshita': {'GitHub Documentation by FreeCodecamp': 4,
                    'ReactJS by lamadev': 4,
                    'JavaScript by freecodecamp': 4,
                    'Basic HTML CSS by code with harry': 3,
                    'NodeJs by Code with Harry': 2},
    
       'Shubhi': {'JavaScript by freecodecamp': 4,
                  'Basic HTML CSS by code with harry': 3,
                  'Web3 by edroh': 5,
                  'NodeJs by Code with Harry': 3},
    
      'Shaurya': {'JavaScript by freecodecamp':4,
                  'NodeJs by Code with Harry':1,
                  'Web3 by edroh':4}}

In [3]:
#create a data frame of this dataset
dataset_df=pd.DataFrame(dataset)
dataset_df.fillna("Not Seen Yet",inplace=True)
dataset_df

,Rahul,Rishabh,Sonali,Ritvik,Harshita,Shubhi,Shaurya
GitHub Documentation by FreeCodecamp,5.0,5.0,2.0,Not Seen Yet,4.0,Not Seen Yet,Not Seen Yet
ReactJS by lamadev,3.0,Not Seen Yet,Not Seen Yet,Not Seen Yet,4.0,Not Seen Yet,Not Seen Yet
NodeJs by Code with Harry,2.0,3.0,Not Seen Yet,Not Seen Yet,2.0,3.0,1.0
Basic HTML CSS by code with harry,3.0,3.0,4.0,4.0,3.0,3.0,Not Seen Yet
JavaScript by freecodecamp,Not Seen Yet,Not Seen Yet,5.0,5.0,4.0,4.0,4.0
Web3 by edroh,Not Seen Yet,Not Seen Yet,3.0,4.0,Not Seen Yet,5.0,4.0


In [4]:
# custom function to create unique set of web series

def unique_items():
    unique_items_list = []
    for person in dataset.keys():
        for items in dataset[person]:
            unique_items_list.append(items)
    s=set(unique_items_list)
    unique_items_list=list(s)
    return unique_items_list

# Cosine Similarity

In [7]:
# custom function to implement cosine similarity between two items i.e. web series

def item_similarity(item1,item2):
    both_rated = {}
    for person in dataset.keys():
        if item1 in dataset[person] and item2 in dataset[person]:
            both_rated[person] = [dataset[person][item1],dataset[person][item2]]

    print(both_rated)
    number_of_ratings = len(both_rated)
    if number_of_ratings == 0:
        return 0

    item1_ratings = [[dataset[k][item1] for k,v in both_rated.items() if item1 in dataset[k] and item2 in dataset[k]]]
    item2_ratings = [[dataset[k][item2] for k, v in both_rated.items() if item1 in dataset[k] and item2 in dataset[k]]]
    print("{} ratings :: {}".format(item1,item1_ratings))
    print("{} ratings :: {}".format(item2,item2_ratings))
    cs = cosine_similarity(item1_ratings,item2_ratings)
    return cs[0][0]

In [8]:
print(item_similarity('GitHub Documentation by FreeCodecamp','JavaScript by freecodecamp'))

{'Sonali': [2, 5], 'Harshita': [4, 4]}
GitHub Documentation by FreeCodecamp ratings :: [[2, 4]]
JavaScript by freecodecamp ratings :: [[5, 4]]
0.9079593845004517


# Phase 1: Item Similarity between multiple items

In [ ]:
#custom function to check most similar items 

def most_similar_items(target_item):
    un_lst=unique_items()
    scores = [(item_similarity(target_item,other_item),target_item+" --> "+other_item) for other_item in un_lst if other_item!=target_item]
    scores.sort(reverse=True)
    return scores

In [ ]:
#custom function to filter the seen videos and unseen videos of the target user

def target_videos_to_users(target_person):
    target_person_video_lst = []
    unique_list =unique_items()
    for videos in dataset[target_person]:
        target_person_video_lst.append(videos)

    s=set(unique_list)
    recommended_videos=list(s.difference(target_person_video_lst))
    a = len(recommended_videos)
    if a == 0:
        return 0
    return recommended_videos,target_person_video_lst

In [ ]:
# function check

unseen_videos,seen_videos=target_videos_to_users('Ritvik')

dct = {"Unseen videos":unseen_videos,"Seen videos":seen_videos}
pd.DataFrame(dct)

# Phase 2: Recommendation Phase :: recommending movies to the target user on behalf of item similarity

In [ ]:
def recommendation_phase(target_person):
    if target_videos_to_users(target_person=target_person) == 0:
#         print(target_person, "has seen all the videos")
        return -1
    not_seen_videos,seen_videos=target_videos_to_users(target_person=target_person)
    seen_ratings = [[dataset[target_person][videos],videos] for videos in dataset[target_person]]
    weighted_avg,weighted_sim = 0,0
    rankings =[]
    for i in not_seen_videos:
        for rate,video in seen_ratings:
            item_sim=item_similarity(i,video)
            weighted_avg +=(item_sim*rate)
            weighted_sim +=item_sim
        weighted_rank=weighted_avg/weighted_sim
        rankings.append([weighted_rank,i])

    rankings.sort(reverse=True)
    return rankings

In [ ]:
print("Enter the target person")
tp = input().title()
if tp in dataset.keys():
    a=recommendation_phase(tp)
    if a != -1:
        print("Recommendation Using Item based Collaborative Filtering:  ")
        for videos,weights in a:
            print(videos," ---> ",weights)
else:
    print("Person not found in the dataset..please try again")